In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

# Libraries to help with data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Libraries to split data, impute missing values
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer

# Libraries to import decision tree classifier and different ensemble classifiers
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor


In [ ]:
test_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')
test_data.head()

In [ ]:
data = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
data.head()

In [ ]:
numerical = data.select_dtypes(include=['number'])

In [ ]:
corr_matrix = numerical.corr()

# Extract the correlation of target variable with other features
target_corr = corr_matrix[['SalePrice']]

In [ ]:
plt.figure(figsize=(8, 12))
sns.heatmap(target_corr,annot=True,vmin=-1,vmax=1);

In [ ]:
train_id = data['Id']
test_id = test_data['Id']

In [ ]:
test_id

In [ ]:
X = data.drop('SalePrice', axis=1)  # Replace 'target_column' with your target
y = data['SalePrice']

In [ ]:
num_features = X.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X.select_dtypes(exclude=[np.number]).columns.tolist()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = encoder.fit_transform(X[cat_features])
test_encoded = encoder.transform(test_data[cat_features])

# Convert encoded categorical features into DataFrame
encoded_feature_names = encoder.get_feature_names_out(cat_features)
X_encoded = pd.DataFrame(X_encoded, columns=encoded_feature_names)
test_encoded = pd.DataFrame(test_encoded, columns=encoded_feature_names)

# Drop original categorical columns and merge encoded ones
X = X.drop(columns=cat_features).reset_index(drop=True)
test_data = test_data.drop(columns=cat_features).reset_index(drop=True)
X = pd.concat([X, X_encoded], axis=1)
test_data = pd.concat([test_data, test_encoded], axis=1)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer

In [ ]:
num_transformer = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),  # More advanced imputation
    ('scaler', StandardScaler()),
    ('power', PowerTransformer(method='yeo-johnson'))
])

In [ ]:
X[num_features] = num_transformer.fit_transform(X[num_features])
test_data[num_features] = num_transformer.transform(test_data[num_features])

# Feature selection model
feature_selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42))
X_selected = feature_selector.fit_transform(X, y)
test_selected = feature_selector.transform(test_data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor

# # Define models
models = []
models.append(("Random Forest", RandomForestRegressor(random_state=1)))
models.append(("Linear Regression", LinearRegression()))
models.append(("Bagging", BaggingRegressor(random_state=1)))
models.append(("Ada Boost", AdaBoostRegressor(random_state=1)))
models.append(("Gradient Boost", GradientBoostingRegressor(random_state=1)))
models.append(("cat Boost",CatBoostRegressor(random_seed=42, verbose=200)))
models.append(("XG Boost", XGBRegressor(random_state=1, eval_metric="rmse")))

results1 = []
names = []

print("\nCross-Validation performance on training dataset:\n")
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

for name, model in models:
    cv_result = cross_val_score(model, X_train, y_train, scoring="r2", cv=kfold)
    results1.append(cv_result)
    names.append(name)
    print(f"{name}: Mean R² = {cv_result.mean():.4f}")

print("\nValidation Performance:\n")
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    print(f"{name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
# rf_tuned = RandomForestRegressor(random_state=1)

# parameters = {
#     'n_estimators': [300, 400, 350],
#     'max_depth': [ 20, 30],
#     'min_samples_split': [ 5, 10],
#     'min_samples_leaf': [2, 4],
#     'bootstrap': [True, False],
#     'max_features': ['sqrt', 'log2', 0.5, 1]
# }

# # Run the grid search
# grid_obj = GridSearchCV(rf_tuned, parameters, scoring='r2',cv=5,n_jobs=-1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# rf_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# rf_tuned.fit(X_train, y_train)

# y_pred = rf_tuned.predict(X_val)
    
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_val, y_pred)
    
# print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
# xgb_tuned = XGBRegressor(random_state=1, eval_metric="rmse")

# parameters = {
#     'n_estimators': [300, 400, 500],
#     'learning_rate': [0.001, 0.01, 0.05],
#     'max_depth': [ 7, 8],
#     'subsample': [0.6, 0.8],
#     'colsample_bytree': [0.6, 0.8]
# }

# # Run the grid search
# grid_obj = GridSearchCV(xgb_tuned, parameters, scoring='r2',cv=5,n_jobs=-1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# xgb_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# xgb_tuned.fit(X_train, y_train)

# y_pred = xgb_tuned.predict(X_val)
    
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_val, y_pred)
    
# print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
# gb_tuned = GradientBoostingRegressor(random_state=1)

# parameters = {
#     'n_estimators': [700,800,900],
#     'learning_rate': [0.001, 0.003, 0.005],
#     'max_depth': [ 10,11],
#     'min_samples_split': [20,25],
#     'subsample': [0.75, 0.85,0.95]
# }

# # Run the grid search
# grid_obj = GridSearchCV(gb_tuned, parameters, scoring='r2',cv=5,n_jobs=-1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Set the clf to the best combination of parameters
# gb_tuned = grid_obj.best_estimator_

# # Fit the best algorithm to the data.
# gb_tuned.fit(X_train, y_train)

# y_pred = gb_tuned.predict(X_val)
    
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_val, y_pred)

# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
catboost_tuned = CatBoostRegressor(random_seed=42, verbose=200, 
                                   learning_rate = 0.03,
                                   n_estimators = 1300, 
                                   depth = 12, 
                                   subsample = 0.8, 
                                   l2_leaf_reg = 5,
                                   boosting_type="Ordered"
                                  )
catboost_tuned.fit(X_train, y_train)

y_pred = catboost_tuned.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
# catboost_tuned = CatBoostRegressor(random_seed=42, verbose=200)

# parameters = {
#     'n_estimators': [1300, 1400,1500],
#     'learning_rate': [0.01, 0.03],
#     'depth': [12,13,14],
#     'l2_leaf_reg': [3, 5],
#     'subsample': [0.8, 0.9]
# },

# grid_obj = GridSearchCV(catboost_tuned,parameters,scoring='r2',cv=5,n_jobs=-1)
# grid_obj = grid_obj.fit(X_train, y_train)

# # Best model
# catboost_tuned = grid_obj.best_estimator_

# # Fit on training data
# catboost_tuned.fit(X_train, y_train)

# # Predict on validation data
# y_pred = catboost_tuned.predict(X_val)

# # Evaluate performance
# rmse = mean_squared_error(y_val, y_pred, squared=False)
# mae = mean_absolute_error(y_val, y_pred)
# r2 = r2_score(y_val, y_pred)

# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

In [ ]:
y_pred = catboost_tuned.predict(test_selected)

In [ ]:
selected_columns = pd.DataFrame({
    'Id': test_id,
    'SalePrice': y_pred
})

In [ ]:
selected_columns.to_csv('selected_columns.csv', index=False)

In [ ]:
from IPython.display import FileLink

# Path to your file
file_path = 'selected_columns.csv'

# Generate and display the download link
FileLink(file_path)